# Tech preamble

In [ ]:
import numpy as np
from dask import array as darr
import xarray as xr
import panel as pn
pn.extension()
from matplotlib import pyplot as plt
import hvplot.xarray
from dask.distributed import Client, LocalCluster

import socket

## Set up cluster or connect to existing one

This should significantly speed up multiple sessions as it will avoid duplication of the compute layer.  In production, we'd want to have an autoscaling cluster that is probably best started separately.

In [ ]:
# parameters
cluster_host = "127.0.0.1"
cluster_port = 12345

In [ ]:
def is_port_in_use(host, port):
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex((host, port)) == 0

In [ ]:
if not is_port_in_use(cluster_host, cluster_port):
    cluster = LocalCluster(
        host=f"tcp://{cluster_host}:{cluster_port}",
        threads_per_worker=1, n_workers=1, memory_limit=1e9
    )
else:
    cluster = f"tcp://{cluster_host}:{cluster_port}"
client = Client(cluster)

In [ ]:
client

## Make up some data

In [ ]:
temp = xr.DataArray(
    darr.random.normal(size=(30, 10, 180, 360), chunks=(10, 1, 60, 60)),
    dims=("t", "z", "y", "x")
)  #.compute()
temp

In [ ]:
temp.nbytes / 1e9

## Define a plot and add to panel

In [ ]:
def hv_plot(data):
    return data.hvplot.line()


def plot_time_series_at(data=temp, x=180, y=90, z=5, view_fn=hv_plot):
    return view_fn(
        data.isel(x=x, y=y)
    )

In [ ]:
pn.interact(
    plot_time_series_at,
    x=pn.widgets.IntSlider(start=0, end=359, step=1, value=180),
    y=pn.widgets.IntSlider(start=0, end=180, step=1, value=90),
    z=pn.widgets.IntSlider(start=0, end=9, step=1, value=5)
).servable()